In [1]:
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess MIDI files
def get_notes(data_folder):
    notes = []
    for file in os.listdir(data_folder):
        if file.endswith(".mid"):
            midi = converter.parse(os.path.join(data_folder, file))
            parts = instrument.partitionByInstrument(midi)
            if parts:  # File has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else:  # File has no instrument parts
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

In [3]:
# Prepare input sequences for the model
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in seq_in])
        network_output.append(note_to_int[seq_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)
    return network_input, network_output

In [4]:
# Prepare input sequences for the model
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in seq_in])
        network_output.append(note_to_int[seq_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)
    return network_input, network_output

In [5]:
# Build the LSTM model
def build_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [6]:
# Generate new music
def generate_notes(model, network_input, pitchnames, n_vocab):
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []
    for note_index in range(500):  # Generate 500 notes
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:]
    return prediction_output

In [10]:
# Convert generated notes to MIDI
def create_midi(prediction_output, output_file="output.mid"):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = [note.Note(int(n)) for n in notes_in_chord]
            for n in notes:
                n.storedInstrument = instrument.Piano()
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

In [12]:
# Main execution
DATA_FOLDER = "/content/drive/MyDrive/beeth"
notes = get_notes(DATA_FOLDER)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)
pitchnames = sorted(set(notes))

model = build_model(network_input, n_vocab)
model.fit(network_input, network_output, epochs=100, batch_size=64)

prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
create_midi(prediction_output)
print("Music generated and saved as output.mid!")

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 1998 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'F\xfcr Elise'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=3, channel=None, data=b'Beethoven F\xfcr Elise'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel

Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 11s 78ms/step - loss: 4.5717
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - loss: 4.1340
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - loss: 4.0635
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - loss: 4.0507
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 3.9609
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 3.9922
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 3.9183
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 3.9154
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - loss: 3.9125
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - loss: 3.8646
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - loss: 3.8047
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - loss: 3.7452
Epoch 13/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - loss: 3.6514
Epoch 14/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - loss: 3.5314
Epoch 15/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/st

In [13]:
from google.colab import files
files.download("output.mid")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>